In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Flatten, AveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [ ]:
# Constants
IMG_SIZE = 227
DATA_PATH = 'D:\\Destination\\dataset'
CATEGORIES = ["New_Front", "New_Rear", "Old_Front", "Old_Rear"]

In [ ]:
# Function to create training data
def create_training_data(data_path, categories):
    training_data = []
    for category in categories:
        path = os.path.join(data_path, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
    return training_data

# Create training data
training_data = create_training_data(DATA_PATH, CATEGORIES)

In [ ]:
# Shuffle and separate features and labels
np.random.shuffle(training_data)
X, y = zip(*training_data)

# Convert to numpy arrays
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255.0
y = np.array(y)

In [ ]:
# Split data into training and validation sets
validation_split = 0.1
split_index = int(len(X) * (1 - validation_split))
train_images, validation_images = X[:split_index], X[split_index:]
train_labels, validation_labels = y[:split_index], y[split_index:]

# Convert labels to categorical format
train_labels = to_categorical(train_labels, num_classes=len(CATEGORIES))
validation_labels = to_categorical(validation_labels, num_classes=len(CATEGORIES))

In [ ]:
# Create TensorFlow datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

In [ ]:
# Define function for image processing in the dataset
def process_images(image, label):
    image = tf.image.per_image_standardization(image)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

In [ ]:
# Apply image processing to datasets
batch_size = 32
train_ds = train_ds.map(process_images).batch(batch_size=batch_size, drop_remainder=True)
validation_ds = validation_ds.map(process_images).batch(batch_size=batch_size, drop_remainder=True)

In [ ]:
# Define the model architecture
base_model = MobileNetV2(input_shape=[IMG_SIZE, IMG_SIZE, 3], weights="imagenet", include_top=False)
x = base_model.output
x = AveragePooling2D(pool_size=(7, 7))(x)
x = Flatten(name="flatten")(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='sigmoid')(x)
x = Dropout(0.5)(x)
output = Dense(len(CATEGORIES), activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

# Train the model
epochs = 10
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

# Save the model and weights
model.save('brake_disc_model.h5')